In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Dot, Lambda
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.layers import dot
import urllib
import collections
import os
import zipfile
import numpy as np
import tensorflow as tf

Using TensorFlow backend.
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [9]:
def download(filename, url, expected_bytes):
    if not os.path.exists(os.path.join(os.path.join(os.path.abspath(''),"data"),filename)):
        os.mkdir(os.path.join(os.path.abspath(''),"data"))
        filename, _ = urllib.request.urlretrieve(url + filename, os.path.join(os.path.join(os.path.abspath(''),"data"),filename))
    statinfo = os.stat(os.path.join(os.path.join(os.path.abspath(''),"data"),filename))
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return os.path.join(os.path.join(os.path.abspath(''),"data"),filename)

In [4]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


In [13]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [5]:
def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = download('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
    del vocabulary
    return data, count, dictionary, reverse_dictionary

In [10]:
vocab_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)

Found and verified C:\Users\yoshi\Documents\GitHub\ML-Playground\word2vec-experiments\data\text8.zip


In [11]:
window_size = 3
vector_dim = 300
epochs = 200000

In [12]:
sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

In [14]:
input_target = Input((1,))
input_context = Input((1,))

W0928 16:53:55.051672 20372 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0928 16:53:55.260038 20372 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [15]:
embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

W0928 16:54:03.050639 20372 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [16]:
# setup a cosine similarity operation which will be output in a secondary model
similarity = dot([target,context],axes=1,normalize=True)

In [17]:
# now perform the dot product operation to get a similarity measure
dot_product = dot([target, context], normalize=False, axes=1)
dot_product = Reshape((1,))(dot_product)
# add the sigmoid output layer
output = Dense(1, activation='sigmoid')(dot_product)
# create the primary training model
model = Model(input=[input_target, input_context], output=output)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys
W0928 16:54:26.935605 20372 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0928 16:54:26.955589 20372 deprecation_wrapper.py:119] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0928 16:54:26.963580 20372 deprecation.py:323] From c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future ver

In [18]:
# create a secondary validation model to run our similarity checks during training
validation_model = Model(input=[input_target, input_context], output=similarity)

c:\users\yoshi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("do...)`
  


In [19]:
class SimilarityCallback:
    def run_sim(self,word):
        valid_word = reverse_dictionary[dictionary[word]]
        top_k = 8  # number of nearest neighbors
        sim = self._get_sim(dictionary[word])
        nearest = (-sim).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in range(top_k):
            close_word = reverse_dictionary[nearest[k]]
            log_str = '%s %s,' % (log_str, close_word)
        print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [21]:
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if cnt % 1000 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))

Iteration 0, loss=0.6809330582618713
Iteration 1000, loss=0.7051646113395691
Iteration 2000, loss=0.6949292421340942
Iteration 3000, loss=1.1329927444458008
Iteration 4000, loss=0.7127837538719177
Iteration 5000, loss=0.7452524304389954
Iteration 6000, loss=0.6771693825721741
Iteration 7000, loss=0.6588912606239319
Iteration 8000, loss=0.6899852752685547
Iteration 9000, loss=0.7537251114845276
Iteration 10000, loss=0.6427937746047974
Iteration 11000, loss=0.6095649003982544
Iteration 12000, loss=0.7932949662208557
Iteration 13000, loss=0.6741223335266113
Iteration 14000, loss=0.6151401400566101
Iteration 15000, loss=0.6587762236595154
Iteration 16000, loss=0.8415100574493408
Iteration 17000, loss=0.7330864667892456
Iteration 18000, loss=0.5995334386825562
Iteration 19000, loss=0.7023196220397949
Iteration 20000, loss=0.7869294285774231
Iteration 21000, loss=0.4649149179458618
Iteration 22000, loss=0.6846593618392944
Iteration 23000, loss=0.6644384264945984
Iteration 24000, loss=0.61485

Iteration 197000, loss=0.33350422978401184
Iteration 198000, loss=1.2639933824539185
Iteration 199000, loss=1.2330348491668701


In [22]:
validation_model.save("validation_model.h5")

In [23]:
model.save("classification_model.h5")